In [1]:
import pytest
import numpy as np
from numpy.testing import assert_allclose

import simpeg.electromagnetics.frequency_domain as fdem
import emg3d

### `INIT`

In [2]:
## Receivers

# recset1 and recset2 have one overlapping receiver
# This means that they CANNOT be used with the same source (duplicate
# receivers)
recset1 = np.array([np.arange(2), np.zeros(2), np.zeros(2)]).T
recset2 = np.array([np.arange(3)+1, np.zeros(3), np.zeros(3)]).T

recset3 = np.array([30, 30, 30]).T

rx_ex1 = fdem.receivers.PointElectricField(
        locations=recset1, component='complex', orientation='x')
rx_ex2 = fdem.receivers.PointElectricField(
        locations=recset2, component='complex', orientation='x')
rx_ey1 = fdem.receivers.PointElectricField(
        locations=recset1, component='complex', orientation='y')
rx_hy1 = fdem.receivers.PointMagneticField(
        locations=recset1, component='complex', orientation='x')

rx_hx3 = fdem.receivers.PointMagneticField(
        locations=recset3, component='complex', orientation='x')

## Sources
wire_src_loc = ([-100, -100, 0], [-100, 100, 0])

# 1b == 1a, to test that the conversion works even so
pts_src_loc1a = (-10, -5, 7)
pts_src_loc1b = (-10, -5, 7)

pts_src_loc2 = (-20, -20, -20)

src_list = []

# pts_src_loc1a for two freqs
for frequency in [1., 2.]:
    src_list.append(fdem.sources.LineCurrent(
        location=pts_src_loc1a, azimuth=90, elevation=0,
        receiver_list=[rx_ex1, rx_ey1, rx_hy1], frequency=frequency))

# Another source with the rx_ex2
src_list.append(fdem.sources.ElectricDipole(
    location=pts_src_loc2, azimuth=0, elevation=0,
    receiver_list=[rx_ex2], frequency=2.))

# 1 extra freq, for only one receiver, wire source
src_list.append(fdem.sources.LineCurrent(
        location=wire_src_loc, receiver_list=[rx_hx3, ],
        frequency=20.))

# pts source for the same frequency
src_list.append(fdem.sources.ElectricDipole(
    location=pts_src_loc1b, azimuth=90, elevation=0,
        receiver_list=[rx_ex2], frequency=20.))

## SimPEG survey and conversion  [test on its own]
simpeg_survey = fdem.Survey(src_list)
emg3d_survey, data_map = emg3d.inversion.simpeg.survey2emg3d(simpeg_survey)

### `test_mapping`

In [4]:
data = np.random.randint(100, 999, survey.count)

# Create map
emg3d_data = np.full(emg3d_survey.shape, np.nan)

# Forward map
emg3d_data[data_map] = data

# Map back
edata = emg3d_data[data_map]

# Check
assert_allclose(data, edata)

NameError: name 'survey' is not defined

In [10]:
rec = emg3d.surveys.txrx_coordinates_to_dict(emg3d.RxElectricPoint, ([0, 1, 2], 0, -800, 0, 0))
src = emg3d.surveys.txrx_coordinates_to_dict(emg3d.TxElectricDipole, ([20, 30, 40, 50], -100, -900, 0, 0))
data = np.ones((4, 3, 2))*np.nan

data[0, 2, 0] = 1
data[1, 2, 0] = 2
data[3, :, 0] = 3, 4, 5

data[1, 2, 1] = 6
data[2, 0, 1] = 7

data[3, 1, 0] = 8
data[1, 1, 1] = 9
data[2, ::2, 0] = 10, 11

survey = emg3d.surveys.Survey(
    sources=src,
    receivers=rec,
    frequencies=[2.0, 100],
    data=data,
)

ssurvey = emg3d.inversion.simpeg.Kernel.survey2simpeg(None, survey)
print(survey.shape, survey.size, survey.count)
survey

(4, 3, 2) 24 10


:: Survey ::

<xarray.Dataset> Size: 384B
Dimensions:   (src: 4, rec: 3, freq: 2)
Coordinates:
  * src       (src) <U6 96B 'TxED-1' 'TxED-2' 'TxED-3' 'TxED-4'
  * rec       (rec) <U6 72B 'RxEP-1' 'RxEP-2' 'RxEP-3'
  * freq      (freq) <U3 24B 'f-1' 'f-2'
Data variables:
    observed  (src, rec, freq) float64 192B nan nan nan nan ... 8.0 nan 5.0 nan
Attributes:
    noise_floor:     None
    relative_error:  None

In [12]:
def data2simpeg(data):
    """Convert an emg3d data-xarray to a SimPEG data array."""
    xx = survey.isfinite.reshape((survey.shape[0], -1), order='F').ravel()
    return data.reshape((survey.shape[0], -1), order='F').ravel()[xx]

def data2emg3d(data):
    """Convert a SimPEG data array to an emg3d data-xarray."""
    xx = survey.isfinite.reshape((survey.shape[0], -1), order='F').ravel()
    out = np.ones(
            survey.size,
            dtype=survey.data.observed.dtype
    )*np.nan
    out[xx] = data
    return out.reshape((survey.shape[0], -1)).reshape(survey.shape, order='F')

In [104]:
dmap

array([False, False,  True, False, False, False, False, False,  True,
       False,  True,  True,  True, False,  True,  True, False, False,
        True,  True,  True, False, False, False])

In [102]:
sind = np.arange(survey.size)[dmap]//survey.shape[1]//survey.shape[2]
find = np.arange(survey.size)[dmap]//survey.shape[1]%survey.shape[2]
rind = np.arange(survey.size)[dmap]%survey.shape[1]

In [103]:
survey.data.observed.data[sind, rind, find]

array([ 1.,  2.,  9.,  6., 10., 11.,  7.,  3.,  8.,  5.])

In [15]:
dmap = survey.isfinite.reshape((survey.shape[0], -1), order='F').ravel()
dmap

array([False, False,  True, False, False, False, False, False,  True,
       False,  True,  True,  True, False,  True,  True, False, False,
        True,  True,  True, False, False, False])

In [80]:
sind = np.array([0, 1, 1, 1, 2, 2, 2, 3, 3, 3])
rind = np.array([2, 2, 1, 2, 0, 2, 0, 0, 1, 2])
find = np.array([0, 0, 1, 1, 0, 0, 1, 0, 0, 0])
survey.data.observed.data[sind, rind, find]

array([ 1.,  2.,  9.,  6., 10., 11.,  7.,  3.,  8.,  5.])

In [61]:
data2simpeg(survey.data.observed.data)

array([ 1.,  2.,  9.,  6., 10., 11.,  7.,  3.,  8.,  5.])

In [72]:
survey.data.observed.data[1, 2, 0]

2.0

In [13]:
data2emg3d(data2simpeg(survey.data.observed.data))

array([[[nan, nan],
        [nan, nan],
        [ 1., nan]],

       [[nan, nan],
        [nan,  9.],
        [ 2.,  6.]],

       [[10.,  7.],
        [nan, nan],
        [11., nan]],

       [[ 3., nan],
        [ 8., nan],
        [ 5., nan]]])

In [14]:
survey.data.observed.data

array([[[nan, nan],
        [nan, nan],
        [ 1., nan]],

       [[nan, nan],
        [nan,  9.],
        [ 2.,  6.]],

       [[10.,  7.],
        [nan, nan],
        [11., nan]],

       [[ 3., nan],
        [ 8., nan],
        [ 5., nan]]])